# runner.ipynb

A notebook to use the methods in `jax_conv_lstm.py`

In [1]:
import torch
from torch_conv_lstm import *

In [2]:
path = "/media/z/Data/datasets/language_network/groupA1_traindataset_256x256.h5"
config = read_config('rnn_config.yaml')

data_loader = TorchDataLoader(path, config['batch_size'], shuffle=True)

In [12]:
datum = next(data_loader._get_iterator())
vision, motor, language, mask, lang_mask = datum
print(f"Vision shape: {vision.shape}. \t Vision type: {type(vision)}")
print(f"Motor shape: {motor.shape}. \t Motor type: {type(motor)}")
print(f"Language shape: {language.shape}. \t Language type: {type(language)}.")
print(f"Mask shape: {mask.shape}. \t Mask type: {type(mask)}.")
print(f"Lang_mask shape: {lang_mask.shape}. \t Lang_mask type: {type(lang_mask)}.")

Vision shape: torch.Size([35, 50, 3, 256, 256]). 	 Vision type: <class 'torch.Tensor'>
Motor shape: torch.Size([35, 50, 60]). 	 Motor type: <class 'torch.Tensor'>
Language shape: torch.Size([35, 5, 20]). 	 Language type: <class 'torch.Tensor'>.
Mask shape: torch.Size([35, 50]). 	 Mask type: <class 'torch.Tensor'>.
Lang_mask shape: torch.Size([35, 5]). 	 Lang_mask type: <class 'torch.Tensor'>.


In [11]:
type(data_loader.dataset.motor[0])

numpy.ndarray

In [14]:
# test conv_lstm cell
inp_channels = 3
hid_size = 128  # desired hidden state size
pad = padding(vision.shape[3],
              hid_size,
              config['s'],
              config['inp_kernel_size'],
              config['kd'])

conv_lstm = ConvLSTMCell(inp_channels,
                         config['h_channels'],
                         config['inp_kernel_size'],
                         config['hid_kernel_size'],
                         inp_stride=config['s'],
                         inp_padding=pad,
                         ik_dilation=config['kd'])

image_batch = vision[:, 0, :, :, :].to(torch.float32)

h, c = conv_lstm.init_hidden_from_normal(vision.shape[0], (hid_size, hid_size))

print(f"h shape: {h.shape}")
print(f"c shape: {c.shape}")

h, c = conv_lstm(image_batch, (h, c))

print(f"h shape: {h.shape}")
print(f"c shape: {c.shape}")


h shape: torch.Size([35, 12, 128, 128])
c shape: torch.Size([35, 12, 128, 128])
inp_convs: torch.Size([35, 48, 128, 128])
hid_convs: torch.Size([35, 48, 128, 128])
ic: torch.Size([35, 12, 128, 128])
h shape: torch.Size([35, 12, 128, 128])
c shape: torch.Size([35, 12, 128, 128])


In [6]:
image_batch.dtype

torch.float64